<a href="https://colab.research.google.com/github/danielmlow/tutorials/blob/main/machine_learning/nested_cross_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Author: Satvik Dixit

# Nested gridsearch cross-validation with group-shuffle split

Todo: use optuna



In [ ]:



def classification_pipeline(embeddings, speakers, labels_list, feature_names):
    np.random.seed(42)
    X = embeddings
    y = labels_list
    groups = speakers
    # Define the hyperparameter grid for GridSearchCV
    param_grid = {'C': [0.1, 1, 10], 'l1_ratio': [0, 0.5, 1]}
    # Outer cross-validation loop
    outer_cv = GroupShuffleSplit(n_splits=2, train_size=0.7, random_state=42)
    outer_scores = []
    feats_array = []
    for train_index, test_index in outer_cv.split(X, y, groups):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = [y[i] for i in train_index], [y[i] for i in test_index]
        # Inner cross-validation loop for hyperparameter tuning
        inner_cv = GroupShuffleSplit(n_splits=3, train_size=0.7, random_state=42)
        grid_search = GridSearchCV(LogisticRegression(solver='saga', penalty = 'elasticnet'), param_grid, cv=inner_cv)
        grid_search.fit(X_train, y_train, groups=[groups[i] for i in train_index])
        # Get the best hyperparameters found in the inner loop
        best_params = grid_search.best_params_
        # Fit the model with the best hyperparameters on the training data
        model = LogisticRegression(solver='saga', penalty = 'elasticnet', **best_params)
        # Standardize X_train and transform X_test
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        model.fit(X_train, y_train)
        # Evaluate the model on the test data
        y_pred = model.predict(X_test)
        score = f1_score(y_test, y_pred, average='weighted')
        outer_scores.append(score)
    # Compute and print the mean F1 score across all outer folds
    mean_f1 = np.mean(outer_scores)
    return model, mean_f1